In [13]:
import pandas as pd
import numpy as np
import pylab as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn import cluster
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin

%matplotlib inline

In [2]:
# Calculate average speed, and use that 
class AvgSpeed(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        if 'Trip_distance' not in X.columns:
            raise KeyError('X Dataframe needs to have column "Trip_distance"')
        self.avg = (y / X['Trip_distance']).mean()
        return self
    def predict(self, X):
        if 'Trip_distance' not in X.columns:
            raise KeyError('X Dataframe needs to have column "Trip_distance"')
        return self.avg * X['Trip_distance']

# Load data

In [3]:
# load training
training = pd.read_csv('../data/Yassir/Train (16).csv').set_index('ID')
training['Timestamp'] = pd.to_datetime(training['Timestamp'])
training.head()

,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA
ID,,,,,,,
000FLWA8,2019-12-04 20:01:50+00:00,3.258,36.777,3.003,36.718,39627,2784
000RGOAM,2019-12-10 22:37:09+00:00,3.087,36.707,3.081,36.727,3918,576
001QSGIH,2019-11-23 20:36:10+00:00,3.144,36.739,3.088,36.742,7265,526
002ACV6R,2019-12-01 05:43:21+00:00,3.239,36.784,3.054,36.763,23350,3130
0039Y7A8,2019-12-17 20:30:20+00:00,2.912,36.707,3.207,36.698,36613,2138


In [4]:
# sample set 
sample_set = pd.read_csv('../data/Yassir/SampleSubmission (13).csv').set_index('ID')
sample_set.head()

,ETA
ID,
000V4BQX,0
003WBC5J,0
004O4X3A,0
006CEI5B,0
009G0M2T,0


In [5]:
# testing data
testing = pd.read_csv('../data/Yassir/Test (8).csv').set_index('ID')
testing['Timestamp'] = pd.to_datetime(testing['Timestamp'])
testing.head()

,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance
ID,,,,,,
000V4BQX,2019-12-21 05:52:37+00:00,2.981,36.688,2.978,36.754,17549
003WBC5J,2019-12-25 21:38:53+00:00,3.032,36.769,3.074,36.751,7532
004O4X3A,2019-12-29 21:30:29+00:00,3.035,36.711,3.010,36.758,10194
006CEI5B,2019-12-31 22:51:57+00:00,2.902,36.738,3.208,36.698,32768
009G0M2T,2019-12-28 21:47:22+00:00,2.860,36.692,2.828,36.696,4513


In [6]:
# weather data
weather_df = pd.read_csv('../data/Yassir/Weather.csv')
weather_df.head()

,date,dewpoint_2m_temperature,maximum_2m_air_temperature,mean_2m_air_temperature,mean_sea_level_pressure,minimum_2m_air_temperature,surface_pressure,total_precipitation,u_component_of_wind_10m,v_component_of_wind_10m
0,2019-11-01,290.630524,296.434662,294.125061,101853.617188,292.503998,100806.351562,0.004297,3.561323,0.941695
1,2019-11-02,289.135284,298.432404,295.551666,101225.164062,293.337921,100187.250000,0.001767,5.318593,3.258237
2,2019-11-03,287.667694,296.612122,295.182831,100806.617188,293.674316,99771.414062,0.000797,8.447649,3.172982
3,2019-11-04,287.634644,297.173737,294.368134,101240.929688,292.376221,100200.843750,0.000393,5.991428,2.236700
4,2019-11-05,286.413788,294.284851,292.496979,101131.750000,289.143066,100088.500000,0.004658,6.962730,2.655364


# Make train and OOT set

In [7]:
# make sure sorted by date
training = training.sort_values('Timestamp')
train_df = training.iloc[:-20000]
oot_df = training.iloc[-20000:]
oot_df.shape,train_df.shape

((20000, 7), (63924, 7))

# AVG Speed Model

In [11]:
# baseline model
split = model_selection.TimeSeriesSplit(n_splits=5)
result= model_selection.cross_validate(AvgSpeed(),train_df, train_df['ETA'],cv=split,
                               scoring='neg_root_mean_squared_error')
result, -np.mean(result['test_score']), np.std(result['test_score'])

({'fit_time': array([0.00819898, 0.00813413, 0.01284361, 0.01280761, 0.01328516]),
  'score_time': array([0.00377679, 0.00445986, 0.00536752, 0.00395322, 0.00427747]),
  'test_score': array([-608.83239483, -619.45547343, -642.25524582, -602.10699022,
         -628.52682296])},
 620.2353854497962,
 14.233041543856139)

In [12]:
# check score on Dev set
reg = AvgSpeed()
reg.fit(train_df, train_df.ETA)
np.sqrt(metrics.mean_squared_error(oot_df.ETA, reg.predict(oot_df)))

632.4052293330434

## Make Submission

In [10]:
# fit model on all data
reg = AvgSpeed()
reg.fit(training, training.ETA)
reg.avg # time / distance

0.10475614041193526

In [11]:
# run test data through
submission = testing.copy()
submission['ETA'] = reg.predict(testing)
submission.head()

,Timestamp,Origin_lat,Origin_lon,Destination_lat,Destination_lon,Trip_distance,ETA
ID,,,,,,,
000V4BQX,2019-12-21 05:52:37+00:00,2.981,36.688,2.978,36.754,17549,1838.365508
003WBC5J,2019-12-25 21:38:53+00:00,3.032,36.769,3.074,36.751,7532,789.023250
004O4X3A,2019-12-29 21:30:29+00:00,3.035,36.711,3.010,36.758,10194,1067.884095
006CEI5B,2019-12-31 22:51:57+00:00,2.902,36.738,3.208,36.698,32768,3432.649209
009G0M2T,2019-12-28 21:47:22+00:00,2.860,36.692,2.828,36.696,4513,472.764462


# Do clustering on coordinates

In [ ]:
    preprocessor = ColumnTransformer(
        transformers=[
            ('ordinal', categorical__ordinal_transformer,col_type['Categorical Ordinal']),
            ('continuous', continuous_transformer, col_type['Continuous']),
        ('dummy',dummy_transformer, col_type['Dummy']),
        ('nominal',categorical__nominal_transformer,col_type['Categorical Nominal'])])

    # create model
    pipeline = Pipeline(steps=[
                          ('preprocessor', preprocessor),
        # imbalance class sampler
                            ('undersampling', all_param['undersampling']),
        # feature selection using linear model
                        ('feature selection', feature_selection.SelectFromModel(ensemble.RandomForestClassifier(),max_features=all_param['max_features'],prefit=False)),
        # classifier
                          ('classifier', all_param['model'])])   

# add weather data and try other models

# make submission


In [ ]:
# save
submission[['ETA']].to_csv('submit_base.csv')